In [ ]:
!pip install --ignore-installed sentencepiece langchain huggingface_hub

In [ ]:
!pip install git+https://github.com/huggingface/accelerate.git

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, logging
import textwrap
import locale
from langchain import HuggingFacePipeline
from transformers import pipeline
from huggingface_hub import snapshot_download
from langchain import PromptTemplate, LLMChain
locale.getpreferredencoding = lambda: "UTF-8"
import torch
import transformers
import torch.nn as nn

In [ ]:
pwd

## Upload model on hf

In [43]:
# rm -r models--sambanovasystems--SambaLingo-Arabic-Base/

rm: cannot remove 'models--sambanovasystems--SambaLingo-Arabic-Base/': No such file or directory


In [ ]:
# model_id = "fireworks-ai/firefunction-v1"

In [ ]:
# model_path = snapshot_download(repo_id=model_id, cache_dir="/workspace/models/")

In [ ]:
# model_path

In [ ]:
# # Reload model in FP16 and merge it with LoRA weights
# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     device_map="auto",
#     trust_remote_code=True,
#     torch_dtype = torch.bfloat16,
# )

In [ ]:
# model

In [ ]:
# # Reload tokenizer to save it
# tokenizer = AutoTokenizer.from_pretrained(model_path, add_eos_token=False, trust_remote_code=True)
# # tokenizer.add_tokens(['<sep>', '<pad>', '<mask>', '<predict>', '<FIM_SUFFIX>', '<FIM_PREFIX>', '<FIM_MIDDLE>','<commit_before>','<commit_msg>','<commit_after>','<jupyter_start>','<jupyter_text>','<jupyter_code>','<jupyter_output>','<empty_output>'], special_tokens=True)
# # tokenizer.pad_token = tokenizer.eos_token
# # tokenizer.add_special_tokens({'bos_token':'<|im_start|>system'},)
# # tokenizer.add_special_tokens({'eos_token':'<|im_end|>'})
# # tokenizer.add_special_tokens({'pad_token': '<|im_end|>'})
# # tokenizer.add_tokens(['<|im_start|>system', '<|im_start|>user','<|im_start|>assistant', '<<SYS>>', '[INST]', '[/INST]'], special_tokens=True)
# print("Model & tokenizer loaded")

In [ ]:
# tokenizer

In [ ]:
# model.push_to_hub("qxsecureserver/tool-selection", max_shard_size=1024, private=True)

In [ ]:
# tokenizer.push_to_hub("qxsecureserver/tool-selection", private=True)

## Check tokenizer

In [44]:
model_path = "01-ai/Yi-6B"
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_path, add_eos_token=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'bos_token':'<s>'},)
tokenizer.add_special_tokens({'eos_token':'</s>'})
tokenizer.add_special_tokens({'unk_token': '<unk>'})
tokenizer.add_tokens(['<|im_start|>', '<|im_start|>system', '<|im_start|>user','<|im_start|>assistant'], special_tokens=True)
print("Model & tokenizer loaded")

Model & tokenizer loaded


In [ ]:
# tokenizer

In [ ]:
# tokenizer.encode('<|im_start|>assistant')

In [ ]:
# input_ids = [ 64002, 144, 3961, 678, 562, 6901, 597, 6058, 14135, 98, 144, 64003, 144, 59572, 2985, 717, 4505, 702, 562, 3113, 593, 59568, 78, 77, 3546, 59568, 98, 909, 648, 717, 4505, 631, 46441, 59568, 98, 59568, 79, 78, 77, 863, 59568, 97, 926, 931, 726, 961, 562, 6126, 593, 59568, 81, 3546, 59568, 98, 981, 620, 567, 2090, 3176, 59568, 100, 3357, 59610, 59575, 6134, 594, 13291, 98, 144, 64004, 144, 3747, 7891, 719, 1911, 97, 666, 1049, 592, 2154, 567, 4731, 593, 9892, 3113, 597, 6126, 594, 567, 4432, 593, 7982, 3176, 597, 2090, 3176, 98, 707, 2090, 3176, 662, 9127, 59604, 620, 567, 3176, 702, 878, 663, 2910, 620, 9582, 6174, 97, 597, 567, 7982, 3176, 662, 4666, 59604, 620, 567, 3176, 702, 878, 648, 620, 4505, 98, 647, 3113, 9450, 950, 567, 11686, 620, 1862, 989, 567, 17857, 97, 597, 562, 6126, 9450, 950, 567, 11686, 620, 5380, 989, 567, 17857, 98, 144, 144, 10349, 59610, 59575, 8408, 567, 2090, 3176, 593, 567, 2985, 659, 842, 59605, 17857, 842, 59604, 98, 144, 144, 35592, 59601, 144, 59594, 707, 2985, 717, 4505, 702, 562, 3113, 593, 59568, 78, 77, 59677, 98, 1117, 2296, 567, 7982, 3176, 717, 59568, 86, 77, 59677, 593, 567, 2090, 3176, 97, 810, 842, 59605, 11686, 762, 17857, 842, 2617, 59568, 77, 98, 86, 77, 842, 59604, 98, 144, 59594, 1393, 567, 2985, 717, 4505, 631, 16490, 98, 59568, 79, 78, 77, 863, 97, 926, 931, 726, 961, 562, 6126, 593, 59568, 81, 59677, 98, 1117, 2296, 567, 952, 7982, 3176, 931, 629, 59568, 78, 77, 81, 59677, 593, 567, 2090, 3176, 97, 810, 842, 59605, 11686, 1184, 59568, 79, 78, 77, 762, 17857, 842, 2617, 59568, 78, 98, 77, 81, 842, 59604, 98, 144, 144, 12356, 97, 1714, 59610, 59575, 1146, 828, 567, 6979, 2084, 632, 567, 1858, 1585, 597, 7891, 631, 567, 2090, 3176, 98, 144, 144, 78, 98, 707, 7982, 3176, 702, 562, 3113, 593, 59568, 78, 77, 59677, 620, 59601, 144, 59580, 59653, 11686, 762, 17857, 842, 2617, 59568, 77, 98, 86, 77, 842, 59651, 144, 144, 79, 98, 707, 7982, 3176, 702, 562, 6126, 593, 59568, 81, 59677, 620, 59601, 144, 59580, 59653, 11686, 1184, 59568, 79, 78, 77, 762, 17857, 842, 2617, 59568, 78, 98, 77, 81, 842, 59651, 144, 144, 17600, 567, 1080, 4700, 3641, 745, 567, 7982, 3176, 702, 562, 3113, 97, 666, 748, 18756, 842, 59605, 11686, 842, 59604, 742, 567, 1080, 4700, 1029, 567, 1669, 4700, 59601, 144, 59580, 59653, 17857, 842, 2617, 59568, 77, 98, 86, 77, 1184, 59568, 79, 78, 77, 762, 17857, 842, 2617, 59568, 78, 98, 77, 81, 842, 59651, 144, 144, 12356, 97, 1714, 59610, 59575, 7891, 631, 842, 59605, 17857, 842, 4591, 144, 59580, 59653, 59568, 77, 98, 86, 77, 9127, 1184, 59568, 79, 78, 77, 762, 59568, 78, 98, 77, 81, 9127, 842, 59651, 144, 59580, 59653, 59568, 79, 78, 77, 762, 59568, 78, 98, 77, 81, 9127, 839, 59568, 77, 98, 86, 77, 9127, 842, 59651, 144, 59580, 59653, 59568, 79, 78, 77, 762, 59568, 77, 98, 78, 81, 9127, 842, 59651, 144, 59580, 59653, 17857, 762, 842, 1193, 59595, 79, 78, 77, 1107, 77, 98, 78, 81, 59596, 842, 59651, 144, 59580, 59653, 17857, 762, 59568, 78, 82, 77, 77, 842, 59651, 144, 144, 38991, 97, 567, 2090, 3176, 593, 567, 2985, 620, 16490, 98, 59568, 78, 82, 77, 77, 98, 144, 144, 3747, 28151, 97, 666, 1316, 567, 1585, 883, 567, 9892, 3113, 597, 6126, 592, 1146, 828, 6979, 639, 16213, 567, 7982, 3176, 592, 567, 2090, 3176, 98, 3468, 48054, 597, 612, 5945, 5940, 97, 666, 978, 1918, 592, 7891, 631, 567, 2090, 3176, 98, 64001, 144, 64003, 144, 59572, 5550, 6556, 21656, 737, 59568, 78, 531, 434, 437, 82, 601, 593, 1034, 6648, 98, 909, 1034, 6648, 2272, 620, 59568, 82, 77, 11218, 97, 981, 787, 629, 567, 6648, 1204, 59568, 79, 1299, 100, 144, 27942, 8324, 59601, 662, 59572, 59604, 59568, 83, 81, 11218, 662, 59588, 59604, 59568, 85, 81, 11218, 662, 59581, 59604, 59568, 84, 79, 11218, 662, 59579, 59604, 59568, 83, 81, 11218, 662, 59569, 59604, 59568, 84, 81, 11218, 144, 64004, 144, 3747, 7891, 719, 1911, 97, 666, 787, 3770, 567, 4731, 593, 18096, 4098, 97, 878, 620, 2555, 592, 567, 1217, 18189, 1719, 2872, 594, 13416, 98, 2364, 562, 11994, 15944, 737, 562, 4677, 9892, 1343, 922, 2906, 97, 648, 620, 1067, 592, 629, 5275, 18096, 761, 705, 26061, 98, 144, 144, 2034, 719, 1611, 97, 567, 5550, 15944, 737, 59568, 78, 59598, 82, 593, 1034, 6648, 1343, 990, 98, 1117, 2296, 639, 1343, 990, 97, 567, 5550, 59610, 59575, 6648, 620, 39066, 737, 562, 4426, 639, 7035, 719, 4098, 98, 4611, 59568, 78, 59598, 82, 620, 567, 4098, 2928, 97, 666, 748, 2939, 719, 659, 562, 4426, 593, 59568, 78, 1184, 59568, 78, 59598, 82, 97, 878, 51086, 592, 59568, 83, 59598, 82, 98, 144, 144, 10349, 59610, 59575, 8408, 567, 3654, 6648, 593, 567, 5550, 659, 842, 59605, 715, 59593, 77, 842, 59604, 97, 878, 620, 1858, 659, 59568, 82, 77, 11218, 98, 3276, 853, 990, 97, 567, 6648, 593, 567, 5550, 842, 59605, 715, 59593, 78, 842, 59604, 787, 629, 842, 59605, 715, 59593, 77, 842, 59604, 2056, 567, 4098, 4426, 98, 3276, 1074, 1299, 97, 567, 6648, 593, 567, 5550, 842, 59605, 715, 59593, 79, 842, 59604, 787, 629, 842, 59605, 715, 59593, 78, 842, 59604, 2056, 567, 4098, 4426, 1320, 98, 144, 144, 10349, 59610, 59575, 10082, 719, 3201, 737, 3201, 59601, 144, 144, 902, 28420, 59568, 78, 26554, 144, 59580, 59653, 715, 59593, 78, 762, 715, 59593, 77, 842, 2617, 842, 1470, 59605, 78, 1184, 842, 1193, 59595, 78, 1107, 82, 712, 1188, 59604, 842, 59651, 144, 59580, 59653, 715, 59593, 78, 762, 59568, 82, 77, 842, 2617, 842, 1193, 59595, 83, 1107, 82, 59596, 842, 59651, 144, 59580, 59653, 715, 59593, 78, 762, 59568, 82, 77, 842, 2617, 59568, 78, 98, 79, 842, 59651, 144, 59580, 59653, 715, 59593, 78, 762, 59568, 83, 77, 842, 1083, 59595, 11218, 59596, 842, 59651, 144, 144, 902, 28420, 59568, 79, 26554, 144, 59580, 59653, 715, 59593, 79, 762, 715, 59593, 78, 842, 2617, 842, 1470, 59605, 78, 1184, 842, 1193, 59595, 78, 1107, 82, 712, 1188, 59604, 842, 59651, 144, 59580, 59653, 715, 59593, 79, 762, 59568, 83, 77, 842, 2617, 842, 1193, 59595, 83, 1107, 82, 59596, 842, 59651, 144, 59580, 59653, 715, 59593, 79, 762, 59568, 83, 77, 842, 2617, 59568, 78, 98, 79, 842, 59651, 144, 59580, 59653, 715, 59593, 79, 762, 59568, 84, 79, 842, 1083, 59595, 11218, 59596, 842, 59651, 144, 144, 38991, 97, 1204, 59568, 79, 1299, 97, 567, 6648, 593, 567, 5550, 787, 629, 59568, 84, 79, 11218, 98, 707, 3633, 3886, 620, 3763, 662, 59581, 59604, 59568, 84, 79, 11218, 98, 64001, 144, 64003, 144, 563, 562, 2062, 593, 1065, 97, 909, 59568, 80, 77, 1065, 978, 1442, 592, 1781, 594, 1343, 6764, 97, 59568, 78, 83, 13206, 1158, 629, 8124, 98, 909, 59568, 83, 77, 1065, 978, 1442, 592, 1781, 594, 562, 6764, 97, 1040, 1284, 13206, 1158, 629, 8124, 100, 144, 27942, 8324, 59601, 662, 59572, 59604, 59568, 83, 662, 59588, 59604, 59568, 85, 662, 59581, 59604, 59568, 78, 77, 662, 59579, 59604, 59568, 78, 79, 662, 59569, 59604, 59568, 78, 81, 1714, 59610, 59575, 771, 14384, 17479, 98, 144, 64004, 144, 3747, 7891, 719, 1911, 97, 666, 1049, 592, 2154, 567, 4731, 593, 11573, 597, 29125, 659, 648, 18335, 592, 25363, 4363, 1029, 3953, 98, 2364, 666, 678, 2622, 639, 59568, 80, 77, 1065, 748, 1356, 59568, 78, 83, 13206, 97, 666, 678, 9783, 1325, 2622, 639, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 620, 567, 1774, 593, 567, 1505, 593, 1065, 923, 6764, 597, 567, 1505, 593, 13206, 98, 59568, 144, 144, 10349, 59610, 59575, 2884, 1310, 567, 1911, 1423, 567, 1926, 5126, 59601, 144, 144, 78, 98, 32690, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 98, 144, 79, 98, 47452, 1040, 1284, 13206, 748, 629, 8124, 909, 59568, 83, 77, 1065, 1781, 594, 1343, 6764, 98, 144, 144, 8308, 9692, 59568, 78, 59601, 32690, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 144, 144, 35592, 639, 59568, 80, 77, 1065, 748, 1356, 59568, 78, 83, 13206, 97, 666, 748, 10082, 567, 2918, 1505, 593, 1065, 737, 50210, 567, 1505, 593, 1065, 923, 6764, 737, 567, 1505, 593, 13206, 59601, 144, 144, 59580, 59653, 842, 1083, 59595, 22015, 1505, 593, 1065, 59596, 762, 842, 1083, 59595, 10839, 593, 1065, 923, 6764, 59596, 842, 2617, 842, 1083, 59595, 10839, 593, 13206, 59596, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 22015, 1505, 593, 1065, 59596, 762, 59568, 80, 77, 842, 2617, 59568, 78, 83, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 22015, 1505, 593, 1065, 59596, 762, 59568, 81, 85, 77, 842, 59651, 144, 144, 8308, 9692, 59568, 79, 59601, 47452, 567, 1505, 593, 13206, 651, 59568, 83, 77, 1065, 923, 6764, 144, 144, 12356, 639, 666, 1082, 926, 678, 59568, 81, 85, 77, 1065, 594, 2918, 97, 666, 1125, 592, 1332, 823, 1040, 1284, 13206, 666, 748, 1356, 909, 1343, 6764, 815, 59568, 83, 77, 1065, 98, 2102, 771, 719, 97, 666, 18260, 567, 2918, 1505, 593, 1065, 737, 567, 1505, 593, 1065, 923, 6764, 59601, 144, 144, 59580, 59653, 842, 1083, 59595, 10839, 593, 13206, 59596, 762, 842, 1193, 992, 1083, 59595, 22015, 1505, 593, 1065, 1241, 992, 1083, 59595, 10839, 593, 1065, 923, 6764, 1241, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 10839, 593, 13206, 59596, 762, 842, 1193, 59595, 81, 85, 77, 1107, 83, 77, 59596, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 10839, 593, 13206, 59596, 762, 59568, 85, 842, 59651, 144, 144, 38991, 97, 909, 59568, 83, 77, 1065, 978, 1442, 592, 1781, 594, 562, 6764, 97, 59568, 85, 13206, 1158, 629, 8124, 98, 144, 144, 8308, 2063, 5257, 144, 144, 1263, 3633, 3886, 620, 662, 59588, 59604, 59568, 85, 98, 1079, 8572, 702, 719, 3195, 737, 1080, 24551, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 597, 1126, 32821, 639, 1505, 737, 567, 952, 1505, 593, 1065, 923, 6764, 592, 1332, 567, 1505, 593, 13206, 639, 1158, 629, 8124, 98, 64001, 144, 64003, 144, 2820, 805, 724, 1442, 1812, 28048, 2465, 33217, 592, 1863, 592, 2891, 98, 141, 9220, 6594, 567, 33217, 594, 11744, 97, 597, 1126, 9649, 972, 823, 592, 1019, 1478, 98, 141, 5361, 1105, 986, 16376, 1019, 10999, 98, 141, 7428, 97, 1105, 6594, 567, 11744, 593, 33217, 594, 853, 9125, 632, 567, 20956, 593, 1019, 1478, 97, 597, 1126, 1105, 7368, 1122, 1029, 567, 2487, 592, 900, 1019, 10999, 98, 141, 5361, 1405, 1105, 7174, 97, 562, 2876, 6337, 2514, 97, 20504, 632, 567, 9125, 593, 33217, 97, 597, 24474, 980, 2832, 593, 567, 9125, 98, 141, 8094, 1060, 805, 724, 7174, 97, 1105, 8189, 828, 2832, 593, 567, 16125, 33217, 97, 878, 978, 728, 15455, 97, 796, 567, 1797, 593, 567, 33217, 978, 13509, 98, 141, 6546, 1284, 33217, 978, 13509, 100, 144, 4148, 666, 1082, 567, 3886, 592, 567, 2198, 2522, 620, 59568, 80, 97, 981, 620, 567, 1830, 593, 9175, 6981, 1812, 100, 144, 64004, 144, 3747, 7891, 719, 1911, 97, 666, 787, 1149, 5031, 22661, 11480, 597, 1703, 562, 2896, 593, 9830, 592, 5641, 567, 1830, 593, 567, 9175, 6981, 654, 59607, 59606, 97, 878, 7035, 567, 1505, 593, 28048, 2465, 33217, 1060, 805, 724, 1442, 98, 1079, 678, 1858, 639, 2832, 593, 567, 33217, 978, 24474, 980, 737, 562, 6337, 97, 597, 593, 1367, 97, 2832, 978, 8189, 828, 1112, 614, 3178, 97, 1436, 567, 1797, 978, 13509, 98, 1079, 678, 962, 1858, 639, 567, 1505, 593, 13509, 33217, 620, 59568, 80, 98, 1079, 787, 1149, 719, 1585, 592, 1146, 828, 663, 4700, 597, 7891, 631, 654, 59607, 59606, 98, 144, 144, 10349, 59610, 59575, 2884, 1310, 567, 1911, 3201, 737, 3201, 59601, 144, 144, 78, 98, 1060, 805, 724, 1442, 654, 59607, 59606, 28048, 2465, 33217, 98, 144, 79, 98, 647, 6337, 24474, 980, 2832, 593, 567, 9125, 97, 810, 567, 1505, 593, 33217, 639, 8142, 620, 23427, 1193, 59595, 78, 1107, 79, 59596, 59607, 59606, 98, 144, 80, 98, 1060, 805, 724, 717, 1918, 592, 4441, 2832, 593, 567, 16125, 33217, 97, 810, 567, 1505, 593, 33217, 9711, 620, 23427, 1193, 59595, 78, 1107, 79, 59596, 842, 2617, 842, 1193, 59595, 78, 1107, 79, 59596, 59607, 762, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 59606, 98, 144, 81, 98, 707, 1505, 593, 33217, 13509, 620, 567, 1505, 593, 16125, 33217, 24143, 567, 1505, 593, 33217, 9711, 97, 878, 620, 23427, 1193, 59595, 78, 1107, 79, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 59606, 98, 144, 144, 12356, 97, 1714, 59610, 59575, 1146, 828, 567, 4700, 1423, 567, 1585, 639, 59568, 80, 33217, 978, 13509, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 78, 1107, 79, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 80, 144, 59580, 59651, 144, 144, 3747, 7891, 719, 4700, 97, 666, 1080, 1049, 592, 13001, 567, 2818, 632, 567, 1999, 2266, 98, 4611, 23427, 1193, 59595, 78, 1107, 79, 59596, 59607, 59606, 620, 567, 1368, 659, 23427, 1193, 59595, 79, 1107, 81, 59596, 59607, 59606, 97, 666, 748, 28499, 567, 4700, 659, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 79, 1107, 81, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 80, 144, 59580, 59651, 144, 144, 41469, 3784, 567, 947, 2818, 97, 666, 900, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 79, 1107, 81, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 144, 59580, 59651, 144, 144, 7428, 915, 4700, 51086, 592, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 80, 144, 59580, 59651, 144, 144, 3747, 1332, 567, 1830, 593, 654, 59607, 59606, 97, 666, 36912, 1480, 7528, 593, 567, 4700, 737, 59568, 81, 592, 41369, 654, 59607, 25156, 144, 144, 59580, 59653, 144, 81, 842, 2617, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 81, 842, 2617, 59568, 80, 144, 59580, 59651, 144, 144, 3355, 51086, 592, 59601, 144, 144, 59580, 59653, 144, 59607, 762, 59568, 78, 79, 144, 59580, 59651, 144, 144, 38991, 97, 1060, 805, 724, 1442, 59568, 78, 79, 28048, 2465, 33217, 98, 64001, 144, 64003, 144, 59600, 43074, 620, 2160, 32419, 597, 5952, 35354, 10461, 592, 648, 98, 1214, 4465, 592, 1150, 1813, 660, 815, 59568, 82, 77, 77, 18086, 631, 10805, 98, 647, 4569, 593, 10309, 815, 59568, 78, 77, 77, 18086, 98, 647, 9478, 593, 35354, 10461, 815, 1812, 18086, 98, 1393, 660, 815, 853, 4569, 593, 10309, 97, 1214, 1206, 1235, 59568, 79, 1439, 988, 593, 35354, 10461, 98, 144, 64004, 144, 3747, 7891, 567, 1858, 1911, 97, 666, 1049, 592, 1332, 567, 1505, 593, 18086, 594, 853, 9478, 593, 35354, 10461, 97, 14923, 659, 842, 59605, 1812, 842, 59604, 97, 1253, 639, 950, 25723, 59587, 815, 853, 4569, 593, 10309, 597, 1074, 1439, 988, 593, 35354, 10461, 97, 660, 4859, 578, 562, 2918, 593, 59568, 82, 77, 77, 18086, 631, 10805, 98, 144, 144, 8308, 9692, 59568, 78, 59601, 26965, 1879, 567, 2549, 2758, 144, 59594, 3394, 2111, 594, 853, 4569, 593, 10309, 59601, 59568, 78, 77, 77, 18086, 144, 59594, 20845, 9330, 18086, 631, 10805, 59601, 59568, 82, 77, 77, 18086, 144, 144, 8308, 9692, 59568, 79, 59601, 7077, 828, 567, 4700, 144, 59600, 43074, 59610, 59575, 10805, 8766, 593, 567, 18086, 742, 853, 4569, 593, 10309, 597, 567, 18086, 742, 1074, 1439, 988, 593, 35354, 10461, 98, 1079, 748, 2939, 719, 659, 663, 4700, 59601, 144, 144, 59580, 59653, 842, 1083, 59595, 13525, 2111, 742, 10309, 59596, 1184, 842, 1083, 59595, 13525, 2111, 742, 35354, 10461, 59596, 762, 842, 1083, 59595, 22015, 18086, 59596, 842, 59651, 144, 144, 35592, 639, 567, 18086, 742, 853, 4569, 593, 10309, 678, 59568, 78, 77, 77, 597, 567, 18086, 742, 35354, 10461, 678, 842, 59605, 59568, 79, 842, 2617, 1812, 842, 59604, 662, 21764, 660, 815, 1074, 1439, 988, 59604, 97, 666, 748, 3469, 59601, 144, 144, 59580, 59653, 59568, 78, 77, 77, 1184, 59568, 79, 59607, 762, 59568, 82, 77, 77, 842, 59651, 144, 144, 8308, 9692, 59568, 80, 59601, 35529, 631, 842, 59605, 1812, 842, 59604, 144, 3747, 1332, 567, 1830, 593, 842, 59605, 1812, 842, 59604, 97, 666, 1049, 592, 41369, 842, 59605, 1812, 842, 59604, 632, 853, 2266, 593, 567, 4700, 98, 1079, 1287, 737, 8780, 24632, 59568, 78, 77, 77, 742, 1480, 7528, 59601, 144, 144, 59580, 59653, 59568, 78, 77, 77, 1184, 59568, 79, 59607, 839, 59568, 78, 77, 77, 762, 59568, 82, 77, 77, 839, 59568, 78, 77, 77, 842, 59651, 144, 144, 10507, 703, 5445, 97, 666, 900, 59601, 144, 144, 59580, 59653, 59568, 79, 59607, 762, 59568, 81, 77, 77, 842, 59651, 144, 144, 12356, 97, 666, 18260, 1480, 7528, 737, 59568, 79, 592, 7891, 631, 842, 59605, 1812, 842, 4591, 144, 144, 59580, 59653, 842, 1193, 59595, 79, 59607, 1107, 79, 59596, 762, 842, 1193, 59595, 81, 77, 77, 1107, 79, 59596, 842, 59651, 144, 144, 3355, 3641, 745, 59601, 144, 144, 59580, 59653, 1812, 762, 59568, 79, 77, 77, 842, 59651, 144, 144, 8308, 2063, 5257, 144, 1263, 1505, 593, 18086, 594, 853, 9478, 593, 35354, 10461, 97, 842, 59605, 1812, 842, 59604, 97, 620, 59568, 79, 77, 77, 18086, 98, 6322, 97, 950, 25723, 59587, 10153, 1074, 1439, 988, 593, 35354, 10461, 592, 893, 4569, 593, 10309, 97, 660, 787, 726, 562, 2918, 593, 59568, 82, 77, 77, 18086, 631, 10805, 97, 659, 9330, 98, 144, 144, 1263, 2806, 3886, 620, 59601, 842, 59605, 1812, 762, 59568, 79, 77, 77, 842, 59604, 18086, 923, 9478, 593, 35354, 10461, 98, 64001, 144, 64003, 144, 655, 567, 2382, 593, 1074, 4117, 620, 59568, 81, 82, 597, 567, 602, 59568, 98, 575, 59568, 98, 580, 597, 610, 59568, 98, 575, 59568, 98, 590, 593, 1154, 4117, 678, 59568, 80, 597, 59568, 78, 77, 77, 4430, 59568, 97, 1126, 567, 2382, 593, 567, 49978, 593, 567, 4117, 620, 5329, 592, 1759, 35529, 648, 651, 13291, 98, 144, 64004, 144, 3747, 7891, 719, 1911, 97, 666, 1049, 592, 2154, 567, 3277, 1397, 1074, 4117, 597, 881, 11155, 5052, 14305, 36817, 662, 12841, 59621, 59604, 597, 2417, 805, 14305, 29402, 662, 7799, 59609, 59604, 98, 707, 1774, 593, 1074, 4117, 620, 5329, 592, 567, 1774, 593, 881, 715, 18372, 597, 750, 14369, 98, 1989, 620, 97, 909, 2086, 59572, 59848, 597, 2086, 59588, 59848, 678, 1074, 4117, 97, 1126, 59601, 144, 144, 59580, 59653, 562, 842, 2617, 582, 762, 715, 18372, 59605, 59572, 97, 582, 59604, 842, 2617, 750, 14369, 59605, 59572, 97, 582, 59604, 842, 59651, 144, 144, 35592, 59601, 144, 59594, 707, 2382, 593, 567, 1074, 4117, 620, 59568, 81, 82, 98, 144, 59594, 707, 715, 18372, 593, 567, 1074, 4117, 620, 59568, 80, 98, 144, 59594, 707, 750, 14369, 593, 567, 1074, 4117, 620, 59568, 78, 77, 77, 98, 144, 144, 10349, 59610, 59575, 8408, 567, 1074, 4117, 659, 2086, 59572, 59848, 597, 2086, 59588, 59848, 98, 1079, 726, 59601, 144, 144, 59580, 59653, 562, 1184, 582, 762, 59568, 81, 82, 842, 59651, 144, 59580, 59653, 715, 18372, 59605, 59572, 97, 582, 59604, 762, 59568, 80, 842, 59651, 144, 59580, 59653, 750, 14369, 59605, 59572, 97, 582, 59604, 762, 59568, 78, 77, 77, 842, 59651, 144, 144, 8269, 567, 3277, 1397, 567, 1774, 593, 1074, 4117, 597, 881, 715, 18372, 597, 750, 14369, 97, 666, 726, 59601, 144, 144, 59580, 59653, 562, 842, 2617, 582, 762, 715, 18372, 59605, 59572, 97, 582, 59604, 842, 2617, 750, 14369, 59605, 59572, 97, 582, 59604, 842, 59651, 144, 59580, 59653, 562, 842, 2617, 582, 762, 59568, 80, 842, 2617, 59568, 78, 77, 77, 842, 59651, 144, 59580, 59653, 562, 842, 2617, 582, 762, 59568, 80, 77, 77, 842, 59651, 144, 144, 12356, 97, 666, 1049, 592, 1332, 567, 2382, 593, 567, 49978, 59575, 593, 567, 4117, 97, 878, 620, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 842, 59651, 144, 144, 3747, 1332, 719, 2382, 97, 666, 748, 1149, 567, 2382, 597, 1774, 593, 567, 4117, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 762, 842, 1193, 59595, 59572, 1184, 582, 1107, 59572, 842, 2617, 582, 59596, 842, 59651, 144, 144, 8272, 47507, 567, 2549, 2758, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 762, 842, 1193, 59595, 81, 82, 1107, 80, 77, 77, 59596, 842, 59651, 144, 144, 10507, 703, 5445, 567, 8658, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 762, 842, 1193, 59595, 80, 1107, 79, 77, 59596, 842, 59651, 144, 144, 38991, 97, 567, 2382, 593, 567, 49978, 59575, 593, 567, 4117, 620, 842, 4062, 1193, 59595, 80, 1107, 79, 77, 712, 59604, 98, 144, 144, 3747, 8180, 719, 1338, 97, 666, 748, 3469, 562, 13291, 6134, 639, 52307, 567, 2382, 593, 567, 49978, 59575, 2084, 632, 567, 1858, 715, 18372, 597, 750, 14369, 59601, 144, 144, 114, 11157, 144, 59697, 13237, 2758, 144, 1946, 59593, 1248, 59593, 46136, 762, 59568, 81, 82, 98, 77, 144, 12841, 59621, 762, 59568, 80, 98, 77, 144, 7799, 59609, 762, 59568, 78, 77, 77, 98, 77, 144, 144, 59697, 32690, 567, 1774, 593, 567, 4117, 144, 8065, 59593, 1248, 59593, 46136, 762, 715, 18372, 1307, 750, 14369, 144, 144, 59697, 32690, 567, 2382, 593, 567, 49978, 59575, 593, 567, 4117, 144, 1946, 59593, 1248, 59593, 4814, 809, 1042, 59575, 762, 2382, 59593, 1248, 59593, 46136, 1788, 1774, 59593, 1248, 59593, 46136, 144, 144, 59697, 17599, 567, 1338, 144, 3416, 59605, 1946, 59593, 1248, 59593, 4814, 809, 1042, 59575, 59604, 144, 114, 144, 144, 44106, 719, 6134, 1206, 1863, 745, 567, 1338, 842, 4062, 1193, 59595, 80, 1107, 79, 77, 712, 59604, 97, 878, 28856, 915, 11637, 10813, 98, 64001, 144 ]

# tokenizer.decode(input_ids)

In [64]:
input_ids = [ 64002, 144, 3961, 678, 562, 6901, 597, 6058, 14135, 98, 144, 64003, 144, 59572, 2985, 717, 4505, 702, 562, 3113, 593, 59568, 78, 77, 3546, 59568, 98, 909, 648, 717, 4505, 631, 46441, 59568, 98, 59568, 79, 78, 77, 863, 59568, 97, 926, 931, 726, 961, 562, 6126, 593, 59568, 81, 3546, 59568, 98, 981, 620, 567, 2090, 3176, 59568, 100, 3357, 59610, 59575, 6134, 594, 13291, 98, 144, 64004, 144, 3747, 7891, 719, 1911, 97, 666, 1049, 592, 2154, 567, 4731, 593, 9892, 3113, 597, 6126, 594, 567, 4432, 593, 7982, 3176, 597, 2090, 3176, 98, 707, 2090, 3176, 662, 9127, 59604, 620, 567, 3176, 702, 878, 663, 2910, 620, 9582, 6174, 97, 597, 567, 7982, 3176, 662, 4666, 59604, 620, 567, 3176, 702, 878, 648, 620, 4505, 98, 647, 3113, 9450, 950, 567, 11686, 620, 1862, 989, 567, 17857, 97, 597, 562, 6126, 9450, 950, 567, 11686, 620, 5380, 989, 567, 17857, 98, 144, 144, 10349, 59610, 59575, 8408, 567, 2090, 3176, 593, 567, 2985, 659, 842, 59605, 17857, 842, 59604, 98, 144, 144, 35592, 59601, 144, 59594, 707, 2985, 717, 4505, 702, 562, 3113, 593, 59568, 78, 77, 59677, 98, 1117, 2296, 567, 7982, 3176, 717, 59568, 86, 77, 59677, 593, 567, 2090, 3176, 97, 810, 842, 59605, 11686, 762, 17857, 842, 2617, 59568, 77, 98, 86, 77, 842, 59604, 98, 144, 59594, 1393, 567, 2985, 717, 4505, 631, 16490, 98, 59568, 79, 78, 77, 863, 97, 926, 931, 726, 961, 562, 6126, 593, 59568, 81, 59677, 98, 1117, 2296, 567, 952, 7982, 3176, 931, 629, 59568, 78, 77, 81, 59677, 593, 567, 2090, 3176, 97, 810, 842, 59605, 11686, 1184, 59568, 79, 78, 77, 762, 17857, 842, 2617, 59568, 78, 98, 77, 81, 842, 59604, 98, 144, 144, 12356, 97, 1714, 59610, 59575, 1146, 828, 567, 6979, 2084, 632, 567, 1858, 1585, 597, 7891, 631, 567, 2090, 3176, 98, 144, 144, 78, 98, 707, 7982, 3176, 702, 562, 3113, 593, 59568, 78, 77, 59677, 620, 59601, 144, 59580, 59653, 11686, 762, 17857, 842, 2617, 59568, 77, 98, 86, 77, 842, 59651, 144, 144, 79, 98, 707, 7982, 3176, 702, 562, 6126, 593, 59568, 81, 59677, 620, 59601, 144, 59580, 59653, 11686, 1184, 59568, 79, 78, 77, 762, 17857, 842, 2617, 59568, 78, 98, 77, 81, 842, 59651, 144, 144, 17600, 567, 1080, 4700, 3641, 745, 567, 7982, 3176, 702, 562, 3113, 97, 666, 748, 18756, 842, 59605, 11686, 842, 59604, 742, 567, 1080, 4700, 1029, 567, 1669, 4700, 59601, 144, 59580, 59653, 17857, 842, 2617, 59568, 77, 98, 86, 77, 1184, 59568, 79, 78, 77, 762, 17857, 842, 2617, 59568, 78, 98, 77, 81, 842, 59651, 144, 144, 12356, 97, 1714, 59610, 59575, 7891, 631, 842, 59605, 17857, 842, 4591, 144, 59580, 59653, 59568, 77, 98, 86, 77, 9127, 1184, 59568, 79, 78, 77, 762, 59568, 78, 98, 77, 81, 9127, 842, 59651, 144, 59580, 59653, 59568, 79, 78, 77, 762, 59568, 78, 98, 77, 81, 9127, 839, 59568, 77, 98, 86, 77, 9127, 842, 59651, 144, 59580, 59653, 59568, 79, 78, 77, 762, 59568, 77, 98, 78, 81, 9127, 842, 59651, 144, 59580, 59653, 17857, 762, 842, 1193, 59595, 79, 78, 77, 1107, 77, 98, 78, 81, 59596, 842, 59651, 144, 59580, 59653, 17857, 762, 59568, 78, 82, 77, 77, 842, 59651, 144, 144, 38991, 97, 567, 2090, 3176, 593, 567, 2985, 620, 16490, 98, 59568, 78, 82, 77, 77, 98, 144, 144, 3747, 28151, 97, 666, 1316, 567, 1585, 883, 567, 9892, 3113, 597, 6126, 592, 1146, 828, 6979, 639, 16213, 567, 7982, 3176, 592, 567, 2090, 3176, 98, 3468, 48054, 597, 612, 5945, 5940, 97, 666, 978, 1918, 592, 7891, 631, 567, 2090, 3176, 98, 64001, 144, 64003, 144, 59572, 5550, 6556, 21656, 737, 59568, 78, 531, 434, 437, 82, 601, 593, 1034, 6648, 98, 909, 1034, 6648, 2272, 620, 59568, 82, 77, 11218, 97, 981, 787, 629, 567, 6648, 1204, 59568, 79, 1299, 100, 144, 27942, 8324, 59601, 662, 59572, 59604, 59568, 83, 81, 11218, 662, 59588, 59604, 59568, 85, 81, 11218, 662, 59581, 59604, 59568, 84, 79, 11218, 662, 59579, 59604, 59568, 83, 81, 11218, 662, 59569, 59604, 59568, 84, 81, 11218, 144, 64004, 144, 3747, 7891, 719, 1911, 97, 666, 787, 3770, 567, 4731, 593, 18096, 4098, 97, 878, 620, 2555, 592, 567, 1217, 18189, 1719, 2872, 594, 13416, 98, 2364, 562, 11994, 15944, 737, 562, 4677, 9892, 1343, 922, 2906, 97, 648, 620, 1067, 592, 629, 5275, 18096, 761, 705, 26061, 98, 144, 144, 2034, 719, 1611, 97, 567, 5550, 15944, 737, 59568, 78, 59598, 82, 593, 1034, 6648, 1343, 990, 98, 1117, 2296, 639, 1343, 990, 97, 567, 5550, 59610, 59575, 6648, 620, 39066, 737, 562, 4426, 639, 7035, 719, 4098, 98, 4611, 59568, 78, 59598, 82, 620, 567, 4098, 2928, 97, 666, 748, 2939, 719, 659, 562, 4426, 593, 59568, 78, 1184, 59568, 78, 59598, 82, 97, 878, 51086, 592, 59568, 83, 59598, 82, 98, 144, 144, 10349, 59610, 59575, 8408, 567, 3654, 6648, 593, 567, 5550, 659, 842, 59605, 715, 59593, 77, 842, 59604, 97, 878, 620, 1858, 659, 59568, 82, 77, 11218, 98, 3276, 853, 990, 97, 567, 6648, 593, 567, 5550, 842, 59605, 715, 59593, 78, 842, 59604, 787, 629, 842, 59605, 715, 59593, 77, 842, 59604, 2056, 567, 4098, 4426, 98, 3276, 1074, 1299, 97, 567, 6648, 593, 567, 5550, 842, 59605, 715, 59593, 79, 842, 59604, 787, 629, 842, 59605, 715, 59593, 78, 842, 59604, 2056, 567, 4098, 4426, 1320, 98, 144, 144, 10349, 59610, 59575, 10082, 719, 3201, 737, 3201, 59601, 144, 144, 902, 28420, 59568, 78, 26554, 144, 59580, 59653, 715, 59593, 78, 762, 715, 59593, 77, 842, 2617, 842, 1470, 59605, 78, 1184, 842, 1193, 59595, 78, 1107, 82, 712, 1188, 59604, 842, 59651, 144, 59580, 59653, 715, 59593, 78, 762, 59568, 82, 77, 842, 2617, 842, 1193, 59595, 83, 1107, 82, 59596, 842, 59651, 144, 59580, 59653, 715, 59593, 78, 762, 59568, 82, 77, 842, 2617, 59568, 78, 98, 79, 842, 59651, 144, 59580, 59653, 715, 59593, 78, 762, 59568, 83, 77, 842, 1083, 59595, 11218, 59596, 842, 59651, 144, 144, 902, 28420, 59568, 79, 26554, 144, 59580, 59653, 715, 59593, 79, 762, 715, 59593, 78, 842, 2617, 842, 1470, 59605, 78, 1184, 842, 1193, 59595, 78, 1107, 82, 712, 1188, 59604, 842, 59651, 144, 59580, 59653, 715, 59593, 79, 762, 59568, 83, 77, 842, 2617, 842, 1193, 59595, 83, 1107, 82, 59596, 842, 59651, 144, 59580, 59653, 715, 59593, 79, 762, 59568, 83, 77, 842, 2617, 59568, 78, 98, 79, 842, 59651, 144, 59580, 59653, 715, 59593, 79, 762, 59568, 84, 79, 842, 1083, 59595, 11218, 59596, 842, 59651, 144, 144, 38991, 97, 1204, 59568, 79, 1299, 97, 567, 6648, 593, 567, 5550, 787, 629, 59568, 84, 79, 11218, 98, 707, 3633, 3886, 620, 3763, 662, 59581, 59604, 59568, 84, 79, 11218, 98, 64001, 144, 64003, 144, 563, 562, 2062, 593, 1065, 97, 909, 59568, 80, 77, 1065, 978, 1442, 592, 1781, 594, 1343, 6764, 97, 59568, 78, 83, 13206, 1158, 629, 8124, 98, 909, 59568, 83, 77, 1065, 978, 1442, 592, 1781, 594, 562, 6764, 97, 1040, 1284, 13206, 1158, 629, 8124, 100, 144, 27942, 8324, 59601, 662, 59572, 59604, 59568, 83, 662, 59588, 59604, 59568, 85, 662, 59581, 59604, 59568, 78, 77, 662, 59579, 59604, 59568, 78, 79, 662, 59569, 59604, 59568, 78, 81, 1714, 59610, 59575, 771, 14384, 17479, 98, 144, 64004, 144, 3747, 7891, 719, 1911, 97, 666, 1049, 592, 2154, 567, 4731, 593, 11573, 597, 29125, 659, 648, 18335, 592, 25363, 4363, 1029, 3953, 98, 2364, 666, 678, 2622, 639, 59568, 80, 77, 1065, 748, 1356, 59568, 78, 83, 13206, 97, 666, 678, 9783, 1325, 2622, 639, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 620, 567, 1774, 593, 567, 1505, 593, 1065, 923, 6764, 597, 567, 1505, 593, 13206, 98, 59568, 144, 144, 10349, 59610, 59575, 2884, 1310, 567, 1911, 1423, 567, 1926, 5126, 59601, 144, 144, 78, 98, 32690, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 98, 144, 79, 98, 47452, 1040, 1284, 13206, 748, 629, 8124, 909, 59568, 83, 77, 1065, 1781, 594, 1343, 6764, 98, 144, 144, 8308, 9692, 59568, 78, 59601, 32690, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 144, 144, 35592, 639, 59568, 80, 77, 1065, 748, 1356, 59568, 78, 83, 13206, 97, 666, 748, 10082, 567, 2918, 1505, 593, 1065, 737, 50210, 567, 1505, 593, 1065, 923, 6764, 737, 567, 1505, 593, 13206, 59601, 144, 144, 59580, 59653, 842, 1083, 59595, 22015, 1505, 593, 1065, 59596, 762, 842, 1083, 59595, 10839, 593, 1065, 923, 6764, 59596, 842, 2617, 842, 1083, 59595, 10839, 593, 13206, 59596, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 22015, 1505, 593, 1065, 59596, 762, 59568, 80, 77, 842, 2617, 59568, 78, 83, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 22015, 1505, 593, 1065, 59596, 762, 59568, 81, 85, 77, 842, 59651, 144, 144, 8308, 9692, 59568, 79, 59601, 47452, 567, 1505, 593, 13206, 651, 59568, 83, 77, 1065, 923, 6764, 144, 144, 12356, 639, 666, 1082, 926, 678, 59568, 81, 85, 77, 1065, 594, 2918, 97, 666, 1125, 592, 1332, 823, 1040, 1284, 13206, 666, 748, 1356, 909, 1343, 6764, 815, 59568, 83, 77, 1065, 98, 2102, 771, 719, 97, 666, 18260, 567, 2918, 1505, 593, 1065, 737, 567, 1505, 593, 1065, 923, 6764, 59601, 144, 144, 59580, 59653, 842, 1083, 59595, 10839, 593, 13206, 59596, 762, 842, 1193, 992, 1083, 59595, 22015, 1505, 593, 1065, 1241, 992, 1083, 59595, 10839, 593, 1065, 923, 6764, 1241, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 10839, 593, 13206, 59596, 762, 842, 1193, 59595, 81, 85, 77, 1107, 83, 77, 59596, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 10839, 593, 13206, 59596, 762, 59568, 85, 842, 59651, 144, 144, 38991, 97, 909, 59568, 83, 77, 1065, 978, 1442, 592, 1781, 594, 562, 6764, 97, 59568, 85, 13206, 1158, 629, 8124, 98, 144, 144, 8308, 2063, 5257, 144, 144, 1263, 3633, 3886, 620, 662, 59588, 59604, 59568, 85, 98, 1079, 8572, 702, 719, 3195, 737, 1080, 24551, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 597, 1126, 32821, 639, 1505, 737, 567, 952, 1505, 593, 1065, 923, 6764, 592, 1332, 567, 1505, 593, 13206, 639, 1158, 629, 8124, 98, 64001, 144, 64003, 144, 2820, 805, 724, 1442, 1812, 28048, 2465, 33217, 592, 1863, 592, 2891, 98, 141, 9220, 6594, 567, 33217, 594, 11744, 97, 597, 1126, 9649, 972, 823, 592, 1019, 1478, 98, 141, 5361, 1105, 986, 16376, 1019, 10999, 98, 141, 7428, 97, 1105, 6594, 567, 11744, 593, 33217, 594, 853, 9125, 632, 567, 20956, 593, 1019, 1478, 97, 597, 1126, 1105, 7368, 1122, 1029, 567, 2487, 592, 900, 1019, 10999, 98, 141, 5361, 1405, 1105, 7174, 97, 562, 2876, 6337, 2514, 97, 20504, 632, 567, 9125, 593, 33217, 97, 597, 24474, 980, 2832, 593, 567, 9125, 98, 141, 8094, 1060, 805, 724, 7174, 97, 1105, 8189, 828, 2832, 593, 567, 16125, 33217, 97, 878, 978, 728, 15455, 97, 796, 567, 1797, 593, 567, 33217, 978, 13509, 98, 141, 6546, 1284, 33217, 978, 13509, 100, 144, 4148, 666, 1082, 567, 3886, 592, 567, 2198, 2522, 620, 59568, 80, 97, 981, 620, 567, 1830, 593, 9175, 6981, 1812, 100, 144, 64004, 144, 3747, 7891, 719, 1911, 97, 666, 787, 1149, 5031, 22661, 11480, 597, 1703, 562, 2896, 593, 9830, 592, 5641, 567, 1830, 593, 567, 9175, 6981, 654, 59607, 59606, 97, 878, 7035, 567, 1505, 593, 28048, 2465, 33217, 1060, 805, 724, 1442, 98, 1079, 678, 1858, 639, 2832, 593, 567, 33217, 978, 24474, 980, 737, 562, 6337, 97, 597, 593, 1367, 97, 2832, 978, 8189, 828, 1112, 614, 3178, 97, 1436, 567, 1797, 978, 13509, 98, 1079, 678, 962, 1858, 639, 567, 1505, 593, 13509, 33217, 620, 59568, 80, 98, 1079, 787, 1149, 719, 1585, 592, 1146, 828, 663, 4700, 597, 7891, 631, 654, 59607, 59606, 98, 144, 144, 10349, 59610, 59575, 2884, 1310, 567, 1911, 3201, 737, 3201, 59601, 144, 144, 78, 98, 1060, 805, 724, 1442, 654, 59607, 59606, 28048, 2465, 33217, 98, 144, 79, 98, 647, 6337, 24474, 980, 2832, 593, 567, 9125, 97, 810, 567, 1505, 593, 33217, 639, 8142, 620, 23427, 1193, 59595, 78, 1107, 79, 59596, 59607, 59606, 98, 144, 80, 98, 1060, 805, 724, 717, 1918, 592, 4441, 2832, 593, 567, 16125, 33217, 97, 810, 567, 1505, 593, 33217, 9711, 620, 23427, 1193, 59595, 78, 1107, 79, 59596, 842, 2617, 842, 1193, 59595, 78, 1107, 79, 59596, 59607, 762, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 59606, 98, 144, 81, 98, 707, 1505, 593, 33217, 13509, 620, 567, 1505, 593, 16125, 33217, 24143, 567, 1505, 593, 33217, 9711, 97, 878, 620, 23427, 1193, 59595, 78, 1107, 79, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 59606, 98, 144, 144, 12356, 97, 1714, 59610, 59575, 1146, 828, 567, 4700, 1423, 567, 1585, 639, 59568, 80, 33217, 978, 13509, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 78, 1107, 79, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 80, 144, 59580, 59651, 144, 144, 3747, 7891, 719, 4700, 97, 666, 1080, 1049, 592, 13001, 567, 2818, 632, 567, 1999, 2266, 98, 4611, 23427, 1193, 59595, 78, 1107, 79, 59596, 59607, 59606, 620, 567, 1368, 659, 23427, 1193, 59595, 79, 1107, 81, 59596, 59607, 59606, 97, 666, 748, 28499, 567, 4700, 659, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 79, 1107, 81, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 80, 144, 59580, 59651, 144, 144, 41469, 3784, 567, 947, 2818, 97, 666, 900, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 79, 1107, 81, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 144, 59580, 59651, 144, 144, 7428, 915, 4700, 51086, 592, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 80, 144, 59580, 59651, 144, 144, 3747, 1332, 567, 1830, 593, 654, 59607, 59606, 97, 666, 36912, 1480, 7528, 593, 567, 4700, 737, 59568, 81, 592, 41369, 654, 59607, 25156, 144, 144, 59580, 59653, 144, 81, 842, 2617, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 81, 842, 2617, 59568, 80, 144, 59580, 59651, 144, 144, 3355, 51086, 592, 59601, 144, 144, 59580, 59653, 144, 59607, 762, 59568, 78, 79, 144, 59580, 59651, 144, 144, 38991, 97, 1060, 805, 724, 1442, 59568, 78, 79, 28048, 2465, 33217, 98, 64001, 144, 64003, 144, 59600, 43074, 620, 2160, 32419, 597, 5952, 35354, 10461, 592, 648, 98, 1214, 4465, 592, 1150, 1813, 660, 815, 59568, 82, 77, 77, 18086, 631, 10805, 98, 647, 4569, 593, 10309, 815, 59568, 78, 77, 77, 18086, 98, 647, 9478, 593, 35354, 10461, 815, 1812, 18086, 98, 1393, 660, 815, 853, 4569, 593, 10309, 97, 1214, 1206, 1235, 59568, 79, 1439, 988, 593, 35354, 10461, 98, 144, 64004, 144, 3747, 7891, 567, 1858, 1911, 97, 666, 1049, 592, 1332, 567, 1505, 593, 18086, 594, 853, 9478, 593, 35354, 10461, 97, 14923, 659, 842, 59605, 1812, 842, 59604, 97, 1253, 639, 950, 25723, 59587, 815, 853, 4569, 593, 10309, 597, 1074, 1439, 988, 593, 35354, 10461, 97, 660, 4859, 578, 562, 2918, 593, 59568, 82, 77, 77, 18086, 631, 10805, 98, 144, 144, 8308, 9692, 59568, 78, 59601, 26965, 1879, 567, 2549, 2758, 144, 59594, 3394, 2111, 594, 853, 4569, 593, 10309, 59601, 59568, 78, 77, 77, 18086, 144, 59594, 20845, 9330, 18086, 631, 10805, 59601, 59568, 82, 77, 77, 18086, 144, 144, 8308, 9692, 59568, 79, 59601, 7077, 828, 567, 4700, 144, 59600, 43074, 59610, 59575, 10805, 8766, 593, 567, 18086, 742, 853, 4569, 593, 10309, 597, 567, 18086, 742, 1074, 1439, 988, 593, 35354, 10461, 98, 1079, 748, 2939, 719, 659, 663, 4700, 59601, 144, 144, 59580, 59653, 842, 1083, 59595, 13525, 2111, 742, 10309, 59596, 1184, 842, 1083, 59595, 13525, 2111, 742, 35354, 10461, 59596, 762, 842, 1083, 59595, 22015, 18086, 59596, 842, 59651, 144, 144, 35592, 639, 567, 18086, 742, 853, 4569, 593, 10309, 678, 59568, 78, 77, 77, 597, 567, 18086, 742, 35354, 10461, 678, 842, 59605, 59568, 79, 842, 2617, 1812, 842, 59604, 662, 21764, 660, 815, 1074, 1439, 988, 59604, 97, 666, 748, 3469, 59601, 144, 144, 59580, 59653, 59568, 78, 77, 77, 1184, 59568, 79, 59607, 762, 59568, 82, 77, 77, 842, 59651, 144, 144, 8308, 9692, 59568, 80, 59601, 35529, 631, 842, 59605, 1812, 842, 59604, 144, 3747, 1332, 567, 1830, 593, 842, 59605, 1812, 842, 59604, 97, 666, 1049, 592, 41369, 842, 59605, 1812, 842, 59604, 632, 853, 2266, 593, 567, 4700, 98, 1079, 1287, 737, 8780, 24632, 59568, 78, 77, 77, 742, 1480, 7528, 59601, 144, 144, 59580, 59653, 59568, 78, 77, 77, 1184, 59568, 79, 59607, 839, 59568, 78, 77, 77, 762, 59568, 82, 77, 77, 839, 59568, 78, 77, 77, 842, 59651, 144, 144, 10507, 703, 5445, 97, 666, 900, 59601, 144, 144, 59580, 59653, 59568, 79, 59607, 762, 59568, 81, 77, 77, 842, 59651, 144, 144, 12356, 97, 666, 18260, 1480, 7528, 737, 59568, 79, 592, 7891, 631, 842, 59605, 1812, 842, 4591, 144, 144, 59580, 59653, 842, 1193, 59595, 79, 59607, 1107, 79, 59596, 762, 842, 1193, 59595, 81, 77, 77, 1107, 79, 59596, 842, 59651, 144, 144, 3355, 3641, 745, 59601, 144, 144, 59580, 59653, 1812, 762, 59568, 79, 77, 77, 842, 59651, 144, 144, 8308, 2063, 5257, 144, 1263, 1505, 593, 18086, 594, 853, 9478, 593, 35354, 10461, 97, 842, 59605, 1812, 842, 59604, 97, 620, 59568, 79, 77, 77, 18086, 98, 6322, 97, 950, 25723, 59587, 10153, 1074, 1439, 988, 593, 35354, 10461, 592, 893, 4569, 593, 10309, 97, 660, 787, 726, 562, 2918, 593, 59568, 82, 77, 77, 18086, 631, 10805, 97, 659, 9330, 98, 144, 144, 1263, 2806, 3886, 620, 59601, 842, 59605, 1812, 762, 59568, 79, 77, 77, 842, 59604, 18086, 923, 9478, 593, 35354, 10461, 98, 64001, 144, 64003, 144, 655, 567, 2382, 593, 1074, 4117, 620, 59568, 81, 82, 597, 567, 602, 59568, 98, 575, 59568, 98, 580, 597, 610, 59568, 98, 575, 59568, 98, 590, 593, 1154, 4117, 678, 59568, 80, 597, 59568, 78, 77, 77, 4430, 59568, 97, 1126, 567, 2382, 593, 567, 49978, 593, 567, 4117, 620, 5329, 592, 1759, 35529, 648, 651, 13291, 98, 144, 64004, 144, 3747, 7891, 719, 1911, 97, 666, 1049, 592, 2154, 567, 3277, 1397, 1074, 4117, 597, 881, 11155, 5052, 14305, 36817, 662, 12841, 59621, 59604, 597, 2417, 805, 14305, 29402, 662, 7799, 59609, 59604, 98, 707, 1774, 593, 1074, 4117, 620, 5329, 592, 567, 1774, 593, 881, 715, 18372, 597, 750, 14369, 98, 1989, 620, 97, 909, 2086, 59572, 59848, 597, 2086, 59588, 59848, 678, 1074, 4117, 97, 1126, 59601, 144, 144, 59580, 59653, 562, 842, 2617, 582, 762, 715, 18372, 59605, 59572, 97, 582, 59604, 842, 2617, 750, 14369, 59605, 59572, 97, 582, 59604, 842, 59651, 144, 144, 35592, 59601, 144, 59594, 707, 2382, 593, 567, 1074, 4117, 620, 59568, 81, 82, 98, 144, 59594, 707, 715, 18372, 593, 567, 1074, 4117, 620, 59568, 80, 98, 144, 59594, 707, 750, 14369, 593, 567, 1074, 4117, 620, 59568, 78, 77, 77, 98, 144, 144, 10349, 59610, 59575, 8408, 567, 1074, 4117, 659, 2086, 59572, 59848, 597, 2086, 59588, 59848, 98, 1079, 726, 59601, 144, 144, 59580, 59653, 562, 1184, 582, 762, 59568, 81, 82, 842, 59651, 144, 59580, 59653, 715, 18372, 59605, 59572, 97, 582, 59604, 762, 59568, 80, 842, 59651, 144, 59580, 59653, 750, 14369, 59605, 59572, 97, 582, 59604, 762, 59568, 78, 77, 77, 842, 59651, 144, 144, 8269, 567, 3277, 1397, 567, 1774, 593, 1074, 4117, 597, 881, 715, 18372, 597, 750, 14369, 97, 666, 726, 59601, 144, 144, 59580, 59653, 562, 842, 2617, 582, 762, 715, 18372, 59605, 59572, 97, 582, 59604, 842, 2617, 750, 14369, 59605, 59572, 97, 582, 59604, 842, 59651, 144, 59580, 59653, 562, 842, 2617, 582, 762, 59568, 80, 842, 2617, 59568, 78, 77, 77, 842, 59651, 144, 59580, 59653, 562, 842, 2617, 582, 762, 59568, 80, 77, 77, 842, 59651, 144, 144, 12356, 97, 666, 1049, 592, 1332, 567, 2382, 593, 567, 49978, 59575, 593, 567, 4117, 97, 878, 620, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 842, 59651, 144, 144, 3747, 1332, 719, 2382, 97, 666, 748, 1149, 567, 2382, 597, 1774, 593, 567, 4117, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 762, 842, 1193, 59595, 59572, 1184, 582, 1107, 59572, 842, 2617, 582, 59596, 842, 59651, 144, 144, 8272, 47507, 567, 2549, 2758, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 762, 842, 1193, 59595, 81, 82, 1107, 80, 77, 77, 59596, 842, 59651, 144, 144, 10507, 703, 5445, 567, 8658, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 762, 842, 1193, 59595, 80, 1107, 79, 77, 59596, 842, 59651, 144, 144, 38991, 97, 567, 2382, 593, 567, 49978, 59575, 593, 567, 4117, 620, 842, 4062, 1193, 59595, 80, 1107, 79, 77, 712, 59604, 98, 144, 144, 3747, 8180, 719, 1338, 97, 666, 748, 3469, 562, 13291, 6134, 639, 52307, 567, 2382, 593, 567, 49978, 59575, 2084, 632, 567, 1858, 715, 18372, 597, 750, 14369, 59601, 144, 144, 114, 11157, 144, 59697, 13237, 2758, 144, 1946, 59593, 1248, 59593, 46136, 762, 59568, 81, 82, 98, 77, 144, 12841, 59621, 762, 59568, 80, 98, 77, 144, 7799, 59609, 762, 59568, 78, 77, 77, 98, 77, 144, 144, 59697, 32690, 567, 1774, 593, 567, 4117, 144, 8065, 59593, 1248, 59593, 46136, 762, 715, 18372, 1307, 750, 14369, 144, 144, 59697, 32690, 567, 2382, 593, 567, 49978, 59575, 593, 567, 4117, 144, 1946, 59593, 1248, 59593, 4814, 809, 1042, 59575, 762, 2382, 59593, 1248, 59593, 46136, 1788, 1774, 59593, 1248, 59593, 46136, 144, 144, 59697, 17599, 567, 1338, 144, 3416, 59605, 1946, 59593, 1248, 59593, 4814, 809, 1042, 59575, 59604, 144, 114, 144, 144, 44106, 719, 6134, 1206, 1863, 745, 567, 1338, 842, 4062, 1193, 59595, 80, 1107, 79, 77, 712, 59604, 97, 878, 28856, 915, 11637, 10813, 98, 64001, 144 ]

labels = [ -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 144, 3747, 7891, 719, 1911, 97, 666, 1049, 592, 2154, 567, 4731, 593, 9892, 3113, 597, 6126, 594, 567, 4432, 593, 7982, 3176, 597, 2090, 3176, 98, 707, 2090, 3176, 662, 9127, 59604, 620, 567, 3176, 702, 878, 663, 2910, 620, 9582, 6174, 97, 597, 567, 7982, 3176, 662, 4666, 59604, 620, 567, 3176, 702, 878, 648, 620, 4505, 98, 647, 3113, 9450, 950, 567, 11686, 620, 1862, 989, 567, 17857, 97, 597, 562, 6126, 9450, 950, 567, 11686, 620, 5380, 989, 567, 17857, 98, 144, 144, 10349, 59610, 59575, 8408, 567, 2090, 3176, 593, 567, 2985, 659, 842, 59605, 17857, 842, 59604, 98, 144, 144, 35592, 59601, 144, 59594, 707, 2985, 717, 4505, 702, 562, 3113, 593, 59568, 78, 77, 59677, 98, 1117, 2296, 567, 7982, 3176, 717, 59568, 86, 77, 59677, 593, 567, 2090, 3176, 97, 810, 842, 59605, 11686, 762, 17857, 842, 2617, 59568, 77, 98, 86, 77, 842, 59604, 98, 144, 59594, 1393, 567, 2985, 717, 4505, 631, 16490, 98, 59568, 79, 78, 77, 863, 97, 926, 931, 726, 961, 562, 6126, 593, 59568, 81, 59677, 98, 1117, 2296, 567, 952, 7982, 3176, 931, 629, 59568, 78, 77, 81, 59677, 593, 567, 2090, 3176, 97, 810, 842, 59605, 11686, 1184, 59568, 79, 78, 77, 762, 17857, 842, 2617, 59568, 78, 98, 77, 81, 842, 59604, 98, 144, 144, 12356, 97, 1714, 59610, 59575, 1146, 828, 567, 6979, 2084, 632, 567, 1858, 1585, 597, 7891, 631, 567, 2090, 3176, 98, 144, 144, 78, 98, 707, 7982, 3176, 702, 562, 3113, 593, 59568, 78, 77, 59677, 620, 59601, 144, 59580, 59653, 11686, 762, 17857, 842, 2617, 59568, 77, 98, 86, 77, 842, 59651, 144, 144, 79, 98, 707, 7982, 3176, 702, 562, 6126, 593, 59568, 81, 59677, 620, 59601, 144, 59580, 59653, 11686, 1184, 59568, 79, 78, 77, 762, 17857, 842, 2617, 59568, 78, 98, 77, 81, 842, 59651, 144, 144, 17600, 567, 1080, 4700, 3641, 745, 567, 7982, 3176, 702, 562, 3113, 97, 666, 748, 18756, 842, 59605, 11686, 842, 59604, 742, 567, 1080, 4700, 1029, 567, 1669, 4700, 59601, 144, 59580, 59653, 17857, 842, 2617, 59568, 77, 98, 86, 77, 1184, 59568, 79, 78, 77, 762, 17857, 842, 2617, 59568, 78, 98, 77, 81, 842, 59651, 144, 144, 12356, 97, 1714, 59610, 59575, 7891, 631, 842, 59605, 17857, 842, 4591, 144, 59580, 59653, 59568, 77, 98, 86, 77, 9127, 1184, 59568, 79, 78, 77, 762, 59568, 78, 98, 77, 81, 9127, 842, 59651, 144, 59580, 59653, 59568, 79, 78, 77, 762, 59568, 78, 98, 77, 81, 9127, 839, 59568, 77, 98, 86, 77, 9127, 842, 59651, 144, 59580, 59653, 59568, 79, 78, 77, 762, 59568, 77, 98, 78, 81, 9127, 842, 59651, 144, 59580, 59653, 17857, 762, 842, 1193, 59595, 79, 78, 77, 1107, 77, 98, 78, 81, 59596, 842, 59651, 144, 59580, 59653, 17857, 762, 59568, 78, 82, 77, 77, 842, 59651, 144, 144, 38991, 97, 567, 2090, 3176, 593, 567, 2985, 620, 16490, 98, 59568, 78, 82, 77, 77, 98, 144, 144, 3747, 28151, 97, 666, 1316, 567, 1585, 883, 567, 9892, 3113, 597, 6126, 592, 1146, 828, 6979, 639, 16213, 567, 7982, 3176, 592, 567, 2090, 3176, 98, 3468, 48054, 597, 612, 5945, 5940, 97, 666, 978, 1918, 592, 7891, 631, 567, 2090, 3176, 98, 64001, 144, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 144, 3747, 7891, 719, 1911, 97, 666, 787, 3770, 567, 4731, 593, 18096, 4098, 97, 878, 620, 2555, 592, 567, 1217, 18189, 1719, 2872, 594, 13416, 98, 2364, 562, 11994, 15944, 737, 562, 4677, 9892, 1343, 922, 2906, 97, 648, 620, 1067, 592, 629, 5275, 18096, 761, 705, 26061, 98, 144, 144, 2034, 719, 1611, 97, 567, 5550, 15944, 737, 59568, 78, 59598, 82, 593, 1034, 6648, 1343, 990, 98, 1117, 2296, 639, 1343, 990, 97, 567, 5550, 59610, 59575, 6648, 620, 39066, 737, 562, 4426, 639, 7035, 719, 4098, 98, 4611, 59568, 78, 59598, 82, 620, 567, 4098, 2928, 97, 666, 748, 2939, 719, 659, 562, 4426, 593, 59568, 78, 1184, 59568, 78, 59598, 82, 97, 878, 51086, 592, 59568, 83, 59598, 82, 98, 144, 144, 10349, 59610, 59575, 8408, 567, 3654, 6648, 593, 567, 5550, 659, 842, 59605, 715, 59593, 77, 842, 59604, 97, 878, 620, 1858, 659, 59568, 82, 77, 11218, 98, 3276, 853, 990, 97, 567, 6648, 593, 567, 5550, 842, 59605, 715, 59593, 78, 842, 59604, 787, 629, 842, 59605, 715, 59593, 77, 842, 59604, 2056, 567, 4098, 4426, 98, 3276, 1074, 1299, 97, 567, 6648, 593, 567, 5550, 842, 59605, 715, 59593, 79, 842, 59604, 787, 629, 842, 59605, 715, 59593, 78, 842, 59604, 2056, 567, 4098, 4426, 1320, 98, 144, 144, 10349, 59610, 59575, 10082, 719, 3201, 737, 3201, 59601, 144, 144, 902, 28420, 59568, 78, 26554, 144, 59580, 59653, 715, 59593, 78, 762, 715, 59593, 77, 842, 2617, 842, 1470, 59605, 78, 1184, 842, 1193, 59595, 78, 1107, 82, 712, 1188, 59604, 842, 59651, 144, 59580, 59653, 715, 59593, 78, 762, 59568, 82, 77, 842, 2617, 842, 1193, 59595, 83, 1107, 82, 59596, 842, 59651, 144, 59580, 59653, 715, 59593, 78, 762, 59568, 82, 77, 842, 2617, 59568, 78, 98, 79, 842, 59651, 144, 59580, 59653, 715, 59593, 78, 762, 59568, 83, 77, 842, 1083, 59595, 11218, 59596, 842, 59651, 144, 144, 902, 28420, 59568, 79, 26554, 144, 59580, 59653, 715, 59593, 79, 762, 715, 59593, 78, 842, 2617, 842, 1470, 59605, 78, 1184, 842, 1193, 59595, 78, 1107, 82, 712, 1188, 59604, 842, 59651, 144, 59580, 59653, 715, 59593, 79, 762, 59568, 83, 77, 842, 2617, 842, 1193, 59595, 83, 1107, 82, 59596, 842, 59651, 144, 59580, 59653, 715, 59593, 79, 762, 59568, 83, 77, 842, 2617, 59568, 78, 98, 79, 842, 59651, 144, 59580, 59653, 715, 59593, 79, 762, 59568, 84, 79, 842, 1083, 59595, 11218, 59596, 842, 59651, 144, 144, 38991, 97, 1204, 59568, 79, 1299, 97, 567, 6648, 593, 567, 5550, 787, 629, 59568, 84, 79, 11218, 98, 707, 3633, 3886, 620, 3763, 662, 59581, 59604, 59568, 84, 79, 11218, 98, 64001, 144, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 144, 3747, 7891, 719, 1911, 97, 666, 1049, 592, 2154, 567, 4731, 593, 11573, 597, 29125, 659, 648, 18335, 592, 25363, 4363, 1029, 3953, 98, 2364, 666, 678, 2622, 639, 59568, 80, 77, 1065, 748, 1356, 59568, 78, 83, 13206, 97, 666, 678, 9783, 1325, 2622, 639, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 620, 567, 1774, 593, 567, 1505, 593, 1065, 923, 6764, 597, 567, 1505, 593, 13206, 98, 59568, 144, 144, 10349, 59610, 59575, 2884, 1310, 567, 1911, 1423, 567, 1926, 5126, 59601, 144, 144, 78, 98, 32690, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 98, 144, 79, 98, 47452, 1040, 1284, 13206, 748, 629, 8124, 909, 59568, 83, 77, 1065, 1781, 594, 1343, 6764, 98, 144, 144, 8308, 9692, 59568, 78, 59601, 32690, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 144, 144, 35592, 639, 59568, 80, 77, 1065, 748, 1356, 59568, 78, 83, 13206, 97, 666, 748, 10082, 567, 2918, 1505, 593, 1065, 737, 50210, 567, 1505, 593, 1065, 923, 6764, 737, 567, 1505, 593, 13206, 59601, 144, 144, 59580, 59653, 842, 1083, 59595, 22015, 1505, 593, 1065, 59596, 762, 842, 1083, 59595, 10839, 593, 1065, 923, 6764, 59596, 842, 2617, 842, 1083, 59595, 10839, 593, 13206, 59596, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 22015, 1505, 593, 1065, 59596, 762, 59568, 80, 77, 842, 2617, 59568, 78, 83, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 22015, 1505, 593, 1065, 59596, 762, 59568, 81, 85, 77, 842, 59651, 144, 144, 8308, 9692, 59568, 79, 59601, 47452, 567, 1505, 593, 13206, 651, 59568, 83, 77, 1065, 923, 6764, 144, 144, 12356, 639, 666, 1082, 926, 678, 59568, 81, 85, 77, 1065, 594, 2918, 97, 666, 1125, 592, 1332, 823, 1040, 1284, 13206, 666, 748, 1356, 909, 1343, 6764, 815, 59568, 83, 77, 1065, 98, 2102, 771, 719, 97, 666, 18260, 567, 2918, 1505, 593, 1065, 737, 567, 1505, 593, 1065, 923, 6764, 59601, 144, 144, 59580, 59653, 842, 1083, 59595, 10839, 593, 13206, 59596, 762, 842, 1193, 992, 1083, 59595, 22015, 1505, 593, 1065, 1241, 992, 1083, 59595, 10839, 593, 1065, 923, 6764, 1241, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 10839, 593, 13206, 59596, 762, 842, 1193, 59595, 81, 85, 77, 1107, 83, 77, 59596, 842, 59651, 144, 144, 59580, 59653, 842, 1083, 59595, 10839, 593, 13206, 59596, 762, 59568, 85, 842, 59651, 144, 144, 38991, 97, 909, 59568, 83, 77, 1065, 978, 1442, 592, 1781, 594, 562, 6764, 97, 59568, 85, 13206, 1158, 629, 8124, 98, 144, 144, 8308, 2063, 5257, 144, 144, 1263, 3633, 3886, 620, 662, 59588, 59604, 59568, 85, 98, 1079, 8572, 702, 719, 3195, 737, 1080, 24551, 567, 2918, 1505, 593, 1065, 594, 567, 2062, 597, 1126, 32821, 639, 1505, 737, 567, 952, 1505, 593, 1065, 923, 6764, 592, 1332, 567, 1505, 593, 13206, 639, 1158, 629, 8124, 98, 64001, 144, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 144, 3747, 7891, 719, 1911, 97, 666, 787, 1149, 5031, 22661, 11480, 597, 1703, 562, 2896, 593, 9830, 592, 5641, 567, 1830, 593, 567, 9175, 6981, 654, 59607, 59606, 97, 878, 7035, 567, 1505, 593, 28048, 2465, 33217, 1060, 805, 724, 1442, 98, 1079, 678, 1858, 639, 2832, 593, 567, 33217, 978, 24474, 980, 737, 562, 6337, 97, 597, 593, 1367, 97, 2832, 978, 8189, 828, 1112, 614, 3178, 97, 1436, 567, 1797, 978, 13509, 98, 1079, 678, 962, 1858, 639, 567, 1505, 593, 13509, 33217, 620, 59568, 80, 98, 1079, 787, 1149, 719, 1585, 592, 1146, 828, 663, 4700, 597, 7891, 631, 654, 59607, 59606, 98, 144, 144, 10349, 59610, 59575, 2884, 1310, 567, 1911, 3201, 737, 3201, 59601, 144, 144, 78, 98, 1060, 805, 724, 1442, 654, 59607, 59606, 28048, 2465, 33217, 98, 144, 79, 98, 647, 6337, 24474, 980, 2832, 593, 567, 9125, 97, 810, 567, 1505, 593, 33217, 639, 8142, 620, 23427, 1193, 59595, 78, 1107, 79, 59596, 59607, 59606, 98, 144, 80, 98, 1060, 805, 724, 717, 1918, 592, 4441, 2832, 593, 567, 16125, 33217, 97, 810, 567, 1505, 593, 33217, 9711, 620, 23427, 1193, 59595, 78, 1107, 79, 59596, 842, 2617, 842, 1193, 59595, 78, 1107, 79, 59596, 59607, 762, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 59606, 98, 144, 81, 98, 707, 1505, 593, 33217, 13509, 620, 567, 1505, 593, 16125, 33217, 24143, 567, 1505, 593, 33217, 9711, 97, 878, 620, 23427, 1193, 59595, 78, 1107, 79, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 59606, 98, 144, 144, 12356, 97, 1714, 59610, 59575, 1146, 828, 567, 4700, 1423, 567, 1585, 639, 59568, 80, 33217, 978, 13509, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 78, 1107, 79, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 80, 144, 59580, 59651, 144, 144, 3747, 7891, 719, 4700, 97, 666, 1080, 1049, 592, 13001, 567, 2818, 632, 567, 1999, 2266, 98, 4611, 23427, 1193, 59595, 78, 1107, 79, 59596, 59607, 59606, 620, 567, 1368, 659, 23427, 1193, 59595, 79, 1107, 81, 59596, 59607, 59606, 97, 666, 748, 28499, 567, 4700, 659, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 79, 1107, 81, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 80, 144, 59580, 59651, 144, 144, 41469, 3784, 567, 947, 2818, 97, 666, 900, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 79, 1107, 81, 59596, 59607, 839, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 144, 59580, 59651, 144, 144, 7428, 915, 4700, 51086, 592, 59601, 144, 144, 59580, 59653, 144, 59580, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 80, 144, 59580, 59651, 144, 144, 3747, 1332, 567, 1830, 593, 654, 59607, 59606, 97, 666, 36912, 1480, 7528, 593, 567, 4700, 737, 59568, 81, 592, 41369, 654, 59607, 25156, 144, 144, 59580, 59653, 144, 81, 842, 2617, 842, 1193, 59595, 78, 1107, 81, 59596, 59607, 762, 59568, 81, 842, 2617, 59568, 80, 144, 59580, 59651, 144, 144, 3355, 51086, 592, 59601, 144, 144, 59580, 59653, 144, 59607, 762, 59568, 78, 79, 144, 59580, 59651, 144, 144, 38991, 97, 1060, 805, 724, 1442, 59568, 78, 79, 28048, 2465, 33217, 98, 64001, 144, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 144, 3747, 7891, 567, 1858, 1911, 97, 666, 1049, 592, 1332, 567, 1505, 593, 18086, 594, 853, 9478, 593, 35354, 10461, 97, 14923, 659, 842, 59605, 1812, 842, 59604, 97, 1253, 639, 950, 25723, 59587, 815, 853, 4569, 593, 10309, 597, 1074, 1439, 988, 593, 35354, 10461, 97, 660, 4859, 578, 562, 2918, 593, 59568, 82, 77, 77, 18086, 631, 10805, 98, 144, 144, 8308, 9692, 59568, 78, 59601, 26965, 1879, 567, 2549, 2758, 144, 59594, 3394, 2111, 594, 853, 4569, 593, 10309, 59601, 59568, 78, 77, 77, 18086, 144, 59594, 20845, 9330, 18086, 631, 10805, 59601, 59568, 82, 77, 77, 18086, 144, 144, 8308, 9692, 59568, 79, 59601, 7077, 828, 567, 4700, 144, 59600, 43074, 59610, 59575, 10805, 8766, 593, 567, 18086, 742, 853, 4569, 593, 10309, 597, 567, 18086, 742, 1074, 1439, 988, 593, 35354, 10461, 98, 1079, 748, 2939, 719, 659, 663, 4700, 59601, 144, 144, 59580, 59653, 842, 1083, 59595, 13525, 2111, 742, 10309, 59596, 1184, 842, 1083, 59595, 13525, 2111, 742, 35354, 10461, 59596, 762, 842, 1083, 59595, 22015, 18086, 59596, 842, 59651, 144, 144, 35592, 639, 567, 18086, 742, 853, 4569, 593, 10309, 678, 59568, 78, 77, 77, 597, 567, 18086, 742, 35354, 10461, 678, 842, 59605, 59568, 79, 842, 2617, 1812, 842, 59604, 662, 21764, 660, 815, 1074, 1439, 988, 59604, 97, 666, 748, 3469, 59601, 144, 144, 59580, 59653, 59568, 78, 77, 77, 1184, 59568, 79, 59607, 762, 59568, 82, 77, 77, 842, 59651, 144, 144, 8308, 9692, 59568, 80, 59601, 35529, 631, 842, 59605, 1812, 842, 59604, 144, 3747, 1332, 567, 1830, 593, 842, 59605, 1812, 842, 59604, 97, 666, 1049, 592, 41369, 842, 59605, 1812, 842, 59604, 632, 853, 2266, 593, 567, 4700, 98, 1079, 1287, 737, 8780, 24632, 59568, 78, 77, 77, 742, 1480, 7528, 59601, 144, 144, 59580, 59653, 59568, 78, 77, 77, 1184, 59568, 79, 59607, 839, 59568, 78, 77, 77, 762, 59568, 82, 77, 77, 839, 59568, 78, 77, 77, 842, 59651, 144, 144, 10507, 703, 5445, 97, 666, 900, 59601, 144, 144, 59580, 59653, 59568, 79, 59607, 762, 59568, 81, 77, 77, 842, 59651, 144, 144, 12356, 97, 666, 18260, 1480, 7528, 737, 59568, 79, 592, 7891, 631, 842, 59605, 1812, 842, 4591, 144, 144, 59580, 59653, 842, 1193, 59595, 79, 59607, 1107, 79, 59596, 762, 842, 1193, 59595, 81, 77, 77, 1107, 79, 59596, 842, 59651, 144, 144, 3355, 3641, 745, 59601, 144, 144, 59580, 59653, 1812, 762, 59568, 79, 77, 77, 842, 59651, 144, 144, 8308, 2063, 5257, 144, 1263, 1505, 593, 18086, 594, 853, 9478, 593, 35354, 10461, 97, 842, 59605, 1812, 842, 59604, 97, 620, 59568, 79, 77, 77, 18086, 98, 6322, 97, 950, 25723, 59587, 10153, 1074, 1439, 988, 593, 35354, 10461, 592, 893, 4569, 593, 10309, 97, 660, 787, 726, 562, 2918, 593, 59568, 82, 77, 77, 18086, 631, 10805, 97, 659, 9330, 98, 144, 144, 1263, 2806, 3886, 620, 59601, 842, 59605, 1812, 762, 59568, 79, 77, 77, 842, 59604, 18086, 923, 9478, 593, 35354, 10461, 98, 64001, 144, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 144, 3747, 7891, 719, 1911, 97, 666, 1049, 592, 2154, 567, 3277, 1397, 1074, 4117, 597, 881, 11155, 5052, 14305, 36817, 662, 12841, 59621, 59604, 597, 2417, 805, 14305, 29402, 662, 7799, 59609, 59604, 98, 707, 1774, 593, 1074, 4117, 620, 5329, 592, 567, 1774, 593, 881, 715, 18372, 597, 750, 14369, 98, 1989, 620, 97, 909, 2086, 59572, 59848, 597, 2086, 59588, 59848, 678, 1074, 4117, 97, 1126, 59601, 144, 144, 59580, 59653, 562, 842, 2617, 582, 762, 715, 18372, 59605, 59572, 97, 582, 59604, 842, 2617, 750, 14369, 59605, 59572, 97, 582, 59604, 842, 59651, 144, 144, 35592, 59601, 144, 59594, 707, 2382, 593, 567, 1074, 4117, 620, 59568, 81, 82, 98, 144, 59594, 707, 715, 18372, 593, 567, 1074, 4117, 620, 59568, 80, 98, 144, 59594, 707, 750, 14369, 593, 567, 1074, 4117, 620, 59568, 78, 77, 77, 98, 144, 144, 10349, 59610, 59575, 8408, 567, 1074, 4117, 659, 2086, 59572, 59848, 597, 2086, 59588, 59848, 98, 1079, 726, 59601, 144, 144, 59580, 59653, 562, 1184, 582, 762, 59568, 81, 82, 842, 59651, 144, 59580, 59653, 715, 18372, 59605, 59572, 97, 582, 59604, 762, 59568, 80, 842, 59651, 144, 59580, 59653, 750, 14369, 59605, 59572, 97, 582, 59604, 762, 59568, 78, 77, 77, 842, 59651, 144, 144, 8269, 567, 3277, 1397, 567, 1774, 593, 1074, 4117, 597, 881, 715, 18372, 597, 750, 14369, 97, 666, 726, 59601, 144, 144, 59580, 59653, 562, 842, 2617, 582, 762, 715, 18372, 59605, 59572, 97, 582, 59604, 842, 2617, 750, 14369, 59605, 59572, 97, 582, 59604, 842, 59651, 144, 59580, 59653, 562, 842, 2617, 582, 762, 59568, 80, 842, 2617, 59568, 78, 77, 77, 842, 59651, 144, 59580, 59653, 562, 842, 2617, 582, 762, 59568, 80, 77, 77, 842, 59651, 144, 144, 12356, 97, 666, 1049, 592, 1332, 567, 2382, 593, 567, 49978, 59575, 593, 567, 4117, 97, 878, 620, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 842, 59651, 144, 144, 3747, 1332, 719, 2382, 97, 666, 748, 1149, 567, 2382, 597, 1774, 593, 567, 4117, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 762, 842, 1193, 59595, 59572, 1184, 582, 1107, 59572, 842, 2617, 582, 59596, 842, 59651, 144, 144, 8272, 47507, 567, 2549, 2758, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 762, 842, 1193, 59595, 81, 82, 1107, 80, 77, 77, 59596, 842, 59651, 144, 144, 10507, 703, 5445, 567, 8658, 59601, 144, 144, 59580, 59653, 842, 1193, 59595, 78, 1107, 59572, 59596, 1184, 842, 1193, 59595, 78, 1107, 59588, 59596, 762, 842, 1193, 59595, 80, 1107, 79, 77, 59596, 842, 59651, 144, 144, 38991, 97, 567, 2382, 593, 567, 49978, 59575, 593, 567, 4117, 620, 842, 4062, 1193, 59595, 80, 1107, 79, 77, 712, 59604, 98, 144, 144, 3747, 8180, 719, 1338, 97, 666, 748, 3469, 562, 13291, 6134, 639, 52307, 567, 2382, 593, 567, 49978, 59575, 2084, 632, 567, 1858, 715, 18372, 597, 750, 14369, 59601, 144, 144, 114, 11157, 144, 59697, 13237, 2758, 144, 1946, 59593, 1248, 59593, 46136, 762, 59568, 81, 82, 98, 77, 144, 12841, 59621, 762, 59568, 80, 98, 77, 144, 7799, 59609, 762, 59568, 78, 77, 77, 98, 77, 144, 144, 59697, 32690, 567, 1774, 593, 567, 4117, 144, 8065, 59593, 1248, 59593, 46136, 762, 715, 18372, 1307, 750, 14369, 144, 144, 59697, 32690, 567, 2382, 593, 567, 49978, 59575, 593, 567, 4117, 144, 1946, 59593, 1248, 59593, 4814, 809, 1042, 59575, 762, 2382, 59593, 1248, 59593, 46136, 1788, 1774, 59593, 1248, 59593, 46136, 144, 144, 59697, 17599, 567, 1338, 144, 3416, 59605, 1946, 59593, 1248, 59593, 4814, 809, 1042, 59575, 59604, 144, 114, 144, 144, 44106, 719, 6134, 1206, 1863, 745, 567, 1338, 842, 4062, 1193, 59595, 80, 1107, 79, 77, 712, 59604, 97, 878, 28856, 915, 11637, 10813, 98, 64001, 144 ]

In [69]:
count = 0
for label in labels:
    if label == -100:
        count += 1
    else:
        break

first_input_ids = input_ids[:count]
rest_input_ids = input_ids[count:]

# Initialize variables
split_input_ids = [[]] # To store the resulting lists
current_count = 0 # To count the consecutive labels
last_label = -100
# Loop through labels to determine splits
index = 0
for label in labels:
    if label == -100:
        if last_label != -100:
            split_input_ids.append([])
        split_input_ids[-1].append(input_ids[index])
        last_label = label
    else:
        if last_label == -100:
            split_input_ids.append([])
        split_input_ids[-1].append(input_ids[index])
        last_label = label
    index += 1

In [ ]:
for split in split_input_ids:
    print(tokenizer.decode(split))
    print("\n\n------------------------------------")

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True)
stream_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    temperature=0.6,
    top_p=0.9,
    do_sample=True,
    top_k=50,
    repetition_penalty=1.15,
    batch_size=8,
    streamer=streamer,
)
stream_llm = HuggingFacePipeline(pipeline=stream_pipeline)
stream_template = f'''User: '''
stream_template += "{query}"
stream_template += f'''Assistant: '''

stream_prompt = PromptTemplate(template=stream_template, input_variables=["query"])
stream_llm_chain = LLMChain(prompt=stream_prompt, llm=stream_llm)

In [ ]:
stream_llm_chain.run('''Who are you? and who made you?''')

In [ ]:
import torch, transformers
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from transformers import AutoModelForCausalLM,AutoTokenizer,LlamaTokenizer

print("Creat tokenizer...")
tokenizer = LlamaTokenizer.from_pretrained(model_path, add_eos_token=False, add_bos_token=False, eos_token='<eod>')
tokenizer.add_tokens(['<sep>', '<pad>', '<mask>', '<predict>', '<FIM_SUFFIX>', '<FIM_PREFIX>', '<FIM_MIDDLE>','<commit_before>','<commit_msg>','<commit_after>','<jupyter_start>','<jupyter_text>','<jupyter_code>','<jupyter_output>','<empty_output>'], special_tokens=True)

print("Creat model...")
model = AutoModelForCausalLM.from_pretrained(model_path,device_map='auto',torch_dtype=torch.bfloat16,trust_remote_code=True)

print(model)


In [ ]:
inputs = tokenizer("2+2=", return_tensors="pt")["input_ids"].to("cuda:0")

In [ ]:
outputs = model.generate(inputs,do_sample=False,max_length=100)

In [ ]:
inputs = tokenizer("2+2=", return_tensors="pt")["input_ids"].to("cuda:0")
outputs = model.generate(inputs,do_sample=False,max_length=100)
print(tokenizer.decode(outputs[0]))

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True)
stream_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    temperature=0.6,
    top_p=0.9,
    do_sample=True,
    top_k=50,
    repetition_penalty=1.15,
    batch_size=8,
    streamer=streamer,
)
stream_llm = HuggingFacePipeline(pipeline=stream_pipeline)
stream_template = "{query}"

stream_prompt = PromptTemplate(template=stream_template, input_variables=["query"])
stream_llm_chain = LLMChain(prompt=stream_prompt, llm=stream_llm)

In [ ]:
stream_llm_chain.run('''25-4*2+3\n we can solve this step by step. \n step 1.''')